# A simple MSIP
This problem is originally from Introduction to Stochastic Programming, John R. Birge, page 308. Extensive solver verifies the optimal value is 2.8. For SDDiP solver, setting binarization precision to 0 does not lead to convergence to the optimal value; setting binarization precision to 1 leads to convergence. 

\begin{align*}
    \min_{\substack{x\geq 0}}~ & -2.5 x_1 - 2 x_2 + \mathbb{E} [4.4 y_1 + 3 y_2]\\
    \textrm{s.t.}~ & 4x_1 + 5x_2 \leq 15, x_1 + x_2 \geq 1.5\\
    & \chi_1 = x_1 + 2 x_2\\
    & \chi_2 = 2x_1 + x_2\\
    & 2y_1 + 3y_2\geq h_1 + \chi_1\\
    & 4y_1 + y_2\geq h_2 + \chi_2\\
    & y_1,y_2\in \mathbb{Z}^+
\end{align*}
where $(h_1,h_2)=
    \begin{cases}
        (-2.8,-1.2), \textrm{w.p.}~ 0.5\\
        (-2,-3), \textrm{w.p.}~ 0.5
    \end{cases}
    $
    
    
            

$\chi_1,\chi_2$ are the state variables and implied bounds for them are $1.5\leq\chi_1\leq 6$, $1.5\leq\chi_2\leq 7.5$

In [1]:
from msppy.msp import MSIP
from msppy.solver import Extensive,SDDiP
from msppy.evaluation import EvaluationTrue
import gurobipy
import numpy
precision = 1
numpy.random.seed(2)
MIP = MSIP(T=2, bound=-10)
for t in range(2):
    m = MIP[t]
    chi_now, chi_past = m.addStateVars(2, name='chi', lb=[1.5,1.5], ub=[6,7.5])
    if t == 0:
        x = m.addVars(2, obj =[-2.5,-2.0], name='x', lb=-gurobipy.GRB.INFINITY)
        slack = m.addVars(2, ub=1/(10**precision))
        m.addConstr(4*x[0] + 5*x[1] <= 15)
        m.addConstr(x[0] + x[1] >= 1.5)
        m.addConstr(chi_now[0] == x[0] + 2*x[1] + slack[0])
        m.addConstr(chi_now[1] == x[1] + 2*x[0] + slack[1])
    else:
        y = m.addVars(2, obj=[4.4,3.0], vtype='I', name='y')
        m.addConstr(
            2*y[0] + 3*y[1] - chi_past[0] >= 0, 
            uncertainty={'rhs': [-2.8,-2]}
        )
        m.addConstr(
            4*y[0] + 1*y[1] - chi_past[1] >= 0, 
            uncertainty = {'rhs': [-1.2, -3]}
        )

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [2]:
print('extensive solver: ', Extensive(MIP).solve(outputFlag=0))

Academic license - for non-commercial use only
extensive solver:  -2.8000000000000016


In [3]:
MIP.binarize(bin_stage=2, precision=precision)
SDDiP(MIP).solve(cuts=['LG'], max_iterations=128)

----------------------------------------------------------------
                   SDDP Solver, Lingquan Ding                   
----------------------------------------------------------------
   Iteration               Bound               Value        Time
----------------------------------------------------------------
           1           -7.914360           -0.575000    0.124460
           2           -5.604792            1.450000    0.092255
           3           -5.136649           -0.250000    0.311867
           4           -4.570907           -1.950000    0.094346
           5           -3.974236           -1.300000    0.215332
           6           -3.925449           -4.350000    0.059303
           7           -3.740368           -1.950000    0.129106
           8           -3.512794           -0.900000    0.335159
           9           -3.426626           -3.400000    0.091463
          10           -3.395699           -2.200000    0.160353
          11           -3

         123           -2.801434           -2.800000    0.087512
         124           -2.801434           -2.800000    0.114835
         125           -2.801434           -2.800000    0.109998
         126           -2.801434           -2.800000    0.090192
         127           -2.801434           -2.800000    0.075691
         128           -2.801434           -2.800000    0.066471
----------------------------------------------------------------
Time: 14.371442556381226 seconds
Algorithm stops since iteration:128 has reached


In [4]:
resultTrue = EvaluationTrue(MIP)
resultTrue.run(n_simulations=100)

[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]
[1.5, 1.5]
[4.0, 5.2]

In [5]:
resultTrue.CI

(-2.799999999999999, -2.799999999999999)